# Hello Tensorflow Serving 

본 노트북에서는 이전 단계에서 생성한 모델을 추론 단계에서 어떻게 Tensorflow Serving을 이용하여 서비스형태로 실행할 수 있는지 살펴봅니다. 

### 모델 준비

이전 단계에서 생성한 모델(S3경로)을 로드합니다. (해당 파일은 이전단계에서 실행한 `code/train.py`에서 165라인의 `model.save(ckpt_dir)`명령을 통해 학습을 실행한 컨테이너 내부에 저장된 후 SageMaker에 의해 S3로 Export 된 파일입니다.)
- 참고 : https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model

In [4]:
%store -r tf_mnist_model_data
tf_mnist_model_data

's3://sagemaker-us-east-1-308961792850/tensorflow/mnist/tensorflow-training-2020-12-13-08-24-31-395/model.tar.gz'

S3에 있는 모델(`model.tar.gz`)을 로컬 환경으로 복사(다운로드)합니다.

In [10]:
!mkdir -p model
!aws s3 cp {tf_mnist_model_data} model/
!tar -zxvf model/model.tar.gz -C model

download: s3://sagemaker-us-east-1-308961792850/tensorflow/mnist/tensorflow-training-2020-12-13-08-24-31-395/model.tar.gz to model/model.tar.gz
00000000/
00000000/variables/
00000000/variables/variables.data-00000-of-00001
00000000/variables/variables.index
00000000/assets/
00000000/saved_model.pb


Tensorflow에서 제공하는 `saved_model_cli` 유틸리티 tool을 이용하여 모델의 세부정보를 살펴보겠습니다.

In [16]:
!saved_model_cli show --dir model/00000000 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
 

### Tensorflow Serving 설치

Tensorflow Serving docker 이미지를 가져옵니다. (Tensorflow Serving을 설치하는 방법은 `apt-get install`을 이용하거나 오픈소스로부터 직접 빌드하는 등 여러가지가 있지만 그 중 Docker를 사용하는 것이 가장 간단합니다.)

- TF Serving 설치가이드 : https://www.tensorflow.org/tfx/serving/setup
- Docker hub tensoflow/serving : https://hub.docker.com/r/tensorflow/serving

In [11]:
!docker pull tensorflow/serving
!docker images

Using default tag: latest
latest: Pulling from tensorflow/serving
Digest: sha256:a94b7e3b0e825350675e83b0c2f2fc28f34be358c34e4126a1d828de899ec44f
Status: Image is up to date for tensorflow/serving:latest
docker.io/tensorflow/serving:latest
REPOSITORY                                                         TAG                 IMAGE ID            CREATED             SIZE
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   2.3-cpu-py37        6ff2bf06924d        2 days ago          3.76GB
tensorflow/serving                                                 latest              e0fe79fbb64f        4 months ago        286MB


### Tensorflow serving 실행 

터미널을 열고 아래 명령을 실행합니다.

```
docker run -t --rm -p 8501:8501 \
    -v `pwd`/model/:/models/MNIST \
    -e MODEL_NAME=MNIST \
    tensorflow/serving &
```


In [12]:
!docker ps

CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS              PORTS                              NAMES
720ef179e093        tensorflow/serving   "/usr/bin/tf_serving…"   27 seconds ago      Up 25 seconds       8500/tcp, 0.0.0.0:8501->8501/tcp   hungry_robinson


Tensorflow Serving은 관리용 API를 함께 제공합니다. `metadata` API를 이용하여 실행중인 모델의 IO 정보를 확인합니다.

- 참고 : https://www.tensorflow.org/tfx/serving/api_rest

In [29]:
import requests

base_url = 'http://localhost:8501'
r = requests.get(f'{base_url}/v1/models/MNIST/metadata')
r.json()

{'model_spec': {'name': 'MNIST', 'signature_name': '', 'version': '0'},
 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'input_1': {'dtype': 'DT_FLOAT',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '28', 'name': ''},
         {'size': '28', 'name': ''},
         {'size': '1', 'name': ''}],
        'unknown_rank': False},
       'name': 'serving_default_input_1:0'}},
     'outputs': {'output_1': {'dtype': 'DT_FLOAT',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '10', 'name': ''}],
        'unknown_rank': False},
       'name': 'StatefulPartitionedCall:0'}},
     'method_name': 'tensorflow/serving/predict'},
    '__saved_model_init_op': {'inputs': {},
     'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID',
       'tensor_shape': {'dim': [], 'unknown_rank': True},
       'name': 'NoOp'}},
     'method_name': ''}}}}}

### 추론 실행

임의의 테스트이미지(28x28)를 생성하고 local에서 실행중인 Docker의 `predict` API를 호출합니다.

In [31]:
import numpy as np
import json

test_data = np.random.rand(1,28,28,1).tolist()
payload = json.dumps({"instances":test_data})

r = requests.post("%s/v1/models/MNIST:predict" % base_url, data=payload)
r.json()

{'predictions': [[-1.4736836,
   -0.647388279,
   2.54512691,
   1.32885027,
   -4.05076408,
   2.11787748,
   -0.382329524,
   0.702581167,
   1.75122082,
   -2.11673284]]}

Tensorflow Serving을 이용하여 학습된 모델을 추론서비스로 만들수 있다는 것을 확인하였습니다. 단, 현재까지 확인한 코드는 기본 기능을 확인하는 수준이었으며 실제 production 환경에서는 보다 복잡한 구성이 필요할 것입니다. (예를 들어 병렬처리, 로깅, 모니터링, Auto-scaling, A/B 테스트 등 성능, 안정성, 운영, 보안 등의 고려 필요)

- 참고 : https://www.tensorflow.org/tfx/serving/serving_config

In [28]:
!docker ps|grep tensorflow/serving | awk '{print $1}'| xargs docker stop

b05003bf6f67
